# _Notebook Jupyter_ 5_NLP_modeloClassificador

# Classificação da Aplicação por aprendizado de máquina

## Importando bibliotecas

In [1]:
import pandas as pd, numpy as np, time
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [2]:
# Data e hora da execução do script
print(f'Código executado em {time.strftime("%d/%m/%Y às %H:%M", time.localtime(time.time()))}')

Código executado em 11/01/2022 às 14:26


### Importando a lista de Aplicações

In [3]:
df_aplicacoes = pd.read_csv(r'./bases/Aplicacoes.csv')

In [4]:
df_aplicacoes.head(2)

,APLICACOES
0,ACELLERA ACX 250F 250
1,ACELLERA FRONTLANDER 500


In [5]:
df_aplicacoes.tail(2)

,APLICACOES
859,ZONGSHEN ZS 125
860,ZONGSHEN ZS 200


In [6]:
df_aplicacoes.shape

(861, 1)

## CountVectorizer

### CountVectorizer do DataSet das Aplicaçãoes

In [7]:
# Criação da função CountVectorizer
cvta = CountVectorizer(strip_accents='ascii', lowercase=True)

In [8]:
X_cvta = cvta.fit_transform(df_aplicacoes['APLICACOES'])

In [9]:
# Criação da função TfidfTransformer
tfia = TfidfTransformer(use_idf=True)

In [10]:
X_tfia = tfia.fit_transform(X_cvta)

## Treinando os Modelos com o DataSet Aplicações

### Definindo os parâmetros

Utilizaremos toda a base no treinamento, pois a intenção é criar uma função de classificação para um dataset onde a classificação é inexistente.

In [11]:
X_train=X_tfia.toarray()
y_train=np.array(df_aplicacoes['APLICACOES'])
y_train1=df_aplicacoes['APLICACOES'].index.to_numpy()

### Modelo LinearSVC

In [12]:
# Criando modelo
clfsvc = LinearSVC()
# Treinamento do modelo
clfsvc.fit(X_tfia, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

#### Função de classificação LinearSVC

A função para utilização do modelo, recebe a descrição filtrada Modelo e retorna a aplicação.


In [13]:
def classificaAplicacaoSVC(modelo):
    novo_cvta = cvta.transform(pd.Series(modelo))
    novo_tfia = tfia.transform(novo_cvta)
    aplicacao = clfsvc.predict(novo_tfia)[0]
    return aplicacao

No final o nosso resultado mostrando (Modelo: descrição filtrada para o modelo e Aplicação: aplicação prevista).<br>

In [14]:
# Lista de exemplos de novos produtos
modelos = ['150 CG HONDA TITAN',
           '125 CARGO CG HONDA TITAN',
           'BIZ C100 HONDA',
           '100 HONDA BIZ',
           '100 BIZ BRAVO HONDA',
           '125 YBR GT YAMAHA',
           '250F TWISTER HONDA']
# Loop for para fazer a predição do departamento de novos produtos
for modelo in modelos:
  print('Modelo:', modelo, ' Aplicação:', classificaAplicacaoSVC(modelo))

Modelo: 150 CG HONDA TITAN  Aplicação: HONDA CG TIT TITAN 125 150 160
Modelo: 125 CARGO CG HONDA TITAN  Aplicação: HONDA CG TIT TITAN 125 150 160
Modelo: BIZ C100 HONDA  Aplicação: HONDA BIZ 100 C100 125 C125
Modelo: 100 HONDA BIZ  Aplicação: HONDA BIZ 100 C100 125 C125
Modelo: 100 BIZ BRAVO HONDA  Aplicação: SHINERAY BRAVO 200
Modelo: 125 YBR GT YAMAHA  Aplicação: SUZUKI GT
Modelo: 250F TWISTER HONDA  Aplicação: HONDA TWISTER CBX 250


### Modelo Multinomial Naive Bayes

In [15]:
# Criando modelo
clfmnb = MultinomialNB()
# Treinamento do modelo
clfmnb.fit(X_train, y_train1)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### Função de classificação

A função para utilização do modelo, recebe a descrição filtrada Modelo e retorna a aplicação.


In [16]:
def classificaAplicacaoMNB(modelo):
    novo_cvta = cvta.transform(pd.Series(modelo))
    novo_tfia = tfia.transform(novo_cvta)
    aplicacao = df_aplicacoes['APLICACOES'][clfmnb.predict(novo_tfia)[0]]
    return aplicacao

No final o nosso resultado mostrando (Modelo: descrição filtrada para o modelo e Aplicação: aplicação prevista).<br>

In [17]:
# Lista de exemplos de novos produtos
modelos = ['150 CG FAN HONDA TITAN',
           '125 CARGO CG HONDA TITAN',
           'BIZ C100 HONDA',
           '100 HONDA BIZ',
           '100 BIZ BRAVO HONDA',
           '125 YBR GT YAMAHA',
           '250F TWISTER HONDA']
# Loop for para fazer a predição do departamento de novos produtos
for modelo in modelos:
  print('Modelo:', modelo, 'Aplicação:', classificaAplicacaoMNB(modelo))

Modelo: 150 CG FAN HONDA TITAN Aplicação: HONDA CG FAN
Modelo: 125 CARGO CG HONDA TITAN Aplicação: HONDA CG TIT TITAN 125 150 160
Modelo: BIZ C100 HONDA Aplicação: HONDA BIZ 100 C100 125 C125
Modelo: 100 HONDA BIZ Aplicação: HONDA BIZ 100 C100 125 C125
Modelo: 100 BIZ BRAVO HONDA Aplicação: HONDA BIZ 100 C100 125 C125
Modelo: 125 YBR GT YAMAHA Aplicação: YAMAHA FACTOR YBR 125 YBR125
Modelo: 250F TWISTER HONDA Aplicação: HONDA TWISTER CBX 250


### Modelo de Regressão Logística

In [18]:
# Criando modelo
clflgr = LogisticRegression(solver='lbfgs',multi_class='multinomial')
# Treinamento do modelo
clflgr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

#### Função de classificação

A função para utilização do modelo, recebe a descrição filtrada Modelo e retorna a aplicação.


In [19]:
def classificaAplicacaoLGR(modelo):
    novo_cvt = cvta.transform(pd.Series(modelo))
    novo_tfi = tfia.transform(novo_cvt)
    aplicacao = clflgr.predict(novo_tfi)[0]
    return aplicacao

No final o nosso resultado mostrando (Modelo: descrição filtrada para o modelo e Aplicação: aplicação prevista).<br>

In [20]:
# Lista de exemplos de novos produtos
modelos = ['150 CG FAN HONDA TITAN',
           '125 CARGO CG HONDA TITAN',
           'BIZ C100 HONDA',
           '100 HONDA BIZ',
           '100 BIZ BRAVO HONDA',
           '125 YBR GT YAMAHA',
           '250F TWISTER HONDA']
# Loop for para fazer a predição do departamento de novos produtos
for modelo in modelos:
    print('Modelo:', modelo, 'Aplicação:', classificaAplicacaoLGR(modelo))

Modelo: 150 CG FAN HONDA TITAN Aplicação: HONDA CG FAN
Modelo: 125 CARGO CG HONDA TITAN Aplicação: LIFAN LF125
Modelo: BIZ C100 HONDA Aplicação: HONDA BIZ 100 C100 125 C125
Modelo: 100 HONDA BIZ Aplicação: LIFAN LF125
Modelo: 100 BIZ BRAVO HONDA Aplicação: LIFAN LF125
Modelo: 125 YBR GT YAMAHA Aplicação: LIFAN LF125
Modelo: 250F TWISTER HONDA Aplicação: LIFAN LF125


## Utilização do modelo

### Carregando dataset

In [21]:
# Importa base de dados com os modelos já determinados para um dataframe
df = pd.read_excel(r'./bases/dataframe_modelos_class0.xlsx')
df.iloc[:,-2:].head()

,Modelo,APLICACAO
0,HONDA CG 150 TIT AN TITAN FAN,HONDA CG TIT TITAN 125 150 160
1,CG HONDA TIT AN 125 CARGO TITAN,HONDA CG TIT TITAN 125 150 160
2,HONDA 125 CG FAN,HONDA CG FAN
3,HONDA BIZ C100,HONDA BIZ 100 C100 125 C125
4,MIRAGE 150,KASINSKI MIRAGE 150 250


In [22]:
# Verifica o tamnanho do dataframe
df.shape

(18237, 28)

### Classificando os modelos do DataSet

In [23]:
# Modelo Linear SVC
ini=time.time()
now = time.strftime("%H:%M:%S", time.localtime(time.time()))
print("Hora de início:" + now)

df['APLICACAOSVC']=df['Modelo'].apply(classificaAplicacaoSVC)

now = time.strftime("%H:%M:%S", time.localtime(time.time()))
fim=time.time()
print("Hora de término:" + str(now))
print("Tempo decorrido: " + str(round((fim-ini),1)) + " segundos.")

Hora de início:14:26:16
Hora de término:14:26:24
Tempo decorrido: 8.6 segundos.


In [24]:
# Modelo Multinomial NB
ini=time.time()
now = time.strftime("%H:%M", time.localtime(time.time()))
print("Hora de início:" + now)

df['APLICACAOMNB']=df['Modelo'].apply(classificaAplicacaoMNB)

now = time.strftime("%H:%M", time.localtime(time.time()))
fim=time.time()
print("Hora de término:" + str(now))
print("Tempo decorrido: " + str(round((fim-ini),1)) + " segundos.")

Hora de início:14:26
Hora de término:14:27
Tempo decorrido: 58.1 segundos.


In [25]:
# Modelo Regressão Logística
ini=time.time()
now = time.strftime("%H:%M", time.localtime(time.time()))
print("Hora de início:" + now)

df['APLICACAOLGR']=df['Modelo'].apply(classificaAplicacaoLGR)

now = time.strftime("%H:%M", time.localtime(time.time()))
fim=time.time()
print("Hora de término:" + str(now))
print("Tempo decorrido: " + str(round((fim-ini),1)) + " segundos.")

Hora de início:14:27
Hora de término:14:28
Tempo decorrido: 71.8 segundos.


In [26]:
df.shape

(18237, 31)

In [27]:
df.iloc[:,-6:].sample(5)

,DESCRICAO,Modelo,APLICACAO,APLICACAOSVC,APLICACAOMNB,APLICACAOLGR
8146,nxr 150 bros esd evol,HONDA BROS EVO NXR 150,HONDA NXR 150 BROS BROZ,HONDA NXR 150 BROS BROZ,HONDA NXR 150 BROS BROZ,HONDA NXR 150 BROS BROZ
10700,racing ybr 125 factor,YAMAHA RACING 125 YBR FACTOR,YAMAHA FACTOR YBR 125 YBR125,YAMAHA FACTOR YBR 125 YBR125,YAMAHA FACTOR YBR 125 YBR125,YAMAHA FACTOR YBR 125 YBR125
13344,nxr 160 bros xre 16,HONDA BROS 160 NXR XRE,HONDA NXR 150 BROS BROZ,HONDA NXR 150 BROS BROZ,HONDA NXR 150 BROS BROZ,LIFAN LF125
5415,k504b c100 biz gulfair,HONDA BIZ C100,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125
479,biz 100,HONDA BIZ 100,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125,LIFAN LF125


## Comparação das classificações

In [28]:
# Quantidade de registros divergentes entre os modelos SVC e Multinomial NB
df[df['APLICACAOSVC']!=df['APLICACAOMNB']].shape

(1112, 31)

In [29]:
# Quantidade de registros divergentes entre os modelos SVC e Regressão Logistica
df[df['APLICACAOSVC']!=df['APLICACAOLGR']].shape

(8132, 31)

In [30]:
# Quantidade de registros divergentes entre os modelos Multinomial NB e Regressão Logistica
df[df['APLICACAOMNB']!=df['APLICACAOLGR']].shape

(7999, 31)

In [31]:
# Quantidade de registros divergentes entre a extração e o modelo SVC
df[df['APLICACAO']!=df['APLICACAOSVC']].shape

(2599, 31)

In [32]:
# Quantidade de registros divergentes entre a extração e o modelo Multinomial NB
df[df['APLICACAO']!=df['APLICACAOMNB']].shape

(2508, 31)

In [33]:
# Quantidade de registros divergentes entre a extração e o modelo de Regressão Logística
df[df['APLICACAO']!=df['APLICACAOLGR']].shape

(8741, 31)

Em virtude do tempo de processamento excessivamente maior e também da enormidade de divergências, decidiu-se por abandonar o uso da modelo de regressão logística.

In [34]:
df.drop('APLICACAOLGR', axis=1, inplace=True)

In [35]:
df[df['APLICACAOSVC']!=df['APLICACAOMNB']].iloc[:5,-5:]

,DESCRICAO,Modelo,APLICACAO,APLICACAOSVC,APLICACAOMNB
11,hunter max 125 allen,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER
16,xt 250 tenere allen,YAMAHA TENERE XT 250,XXX,YAMAHA XTZ TENERE 250,YAMAHA XT TENERE 660Z 660
17,yes intruder 125 allen,SUZUKI 125 YES INTRUDER,SUZUKI YES EN 125,SUZUKI INTRUDER,SUZUKI YES EN 125
22,jet 49cc allen,JET,"['SHINERAY JET 125', 'SHINERAY JET 50']",SHINERAY JET 50,SHINERAY JET 125
42,xtz 250 allen,YAMAHA XTZ 250,"['YAMAHA LANDER XTZ 250', 'YAMAHA XTZ TENERE 2...",YAMAHA XTZ 125,YAMAHA XTZ TENERE 250


### Melhorando a classificação

Para chegar à classificação final a ser utilizada na função de classificação, utilizaremos algumas regras:<br>
1. SE APLICACAOMNB==APLICACAOSVC ==> APLICACAOFIM=APLICACAOSVC
2. SE APLICACAOMNB!=APLICACAOSVC
  1. SE APLICACAO=='XXX' ==> APLICACAOFIM=[APLICACAOSVC,APLICACAOMNB]
  2. SE APLICACAO==APLICACAOSVC ==> APLICACAOFIM=APLICACAOSVC
  3. SE APLICACAO==APLICACAOMNB ==> APLICACAOFIM=APLICACAOMNB
  4. SE APLICACAO for uma lista:
    * SE APLICACAOSVC estiver na lista ==> APLICACAOFIM=APLICACAOSVC
    * SE APLICACAOMNB estiver na lista ==> APLICACAOFIM=APLICACAOSMNB
    * SENÃO ==> APLICAFIM=LISTA+[APLICACAOSVC,APLICACAOMNB]
  5. SE APLICACAO!=APLICACAOSVC!=APLICACAOMNB ==> APLICACAOFIM=[APLICACAO,APLICACAOSVC,APLICACAOMNB]

In [36]:
df=df.assign(APLICACAOFIM=df.APLICACAOMNB.tolist())
df.shape

(18237, 31)

In [37]:
df.iloc[:,-5:].head()

,Modelo,APLICACAO,APLICACAOSVC,APLICACAOMNB,APLICACAOFIM
0,HONDA CG 150 TIT AN TITAN FAN,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160
1,CG HONDA TIT AN 125 CARGO TITAN,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160
2,HONDA 125 CG FAN,HONDA CG FAN,HONDA CG FAN,HONDA CG FAN,HONDA CG FAN
3,HONDA BIZ C100,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125
4,MIRAGE 150,KASINSKI MIRAGE 150 250,KASINSKI MIRAGE 150 250,KASINSKI MIRAGE 150 250,KASINSKI MIRAGE 150 250


In [38]:
colindex = df.columns.get_loc("APLICACAOFIM")
for i in range(df.shape[0]):
    #print(i)
    # se APLICACAOSVC==APLICACAOMNB ==> já aplicado pelo df.assign
    # se APLICACAOSVC!=APLICACAOMNB
    if df['APLICACAOSVC'][i]!=df['APLICACAOMNB'][i]:
        # se APLICAÇÃO for sem valor ('XXX') ==> APLICACAOFIM=[APLICACAOSVC,APLICACAOMNB]
        if df['APLICACAO'][i]=='XXX':
            # APLICACAOFIM será a lista com os dois valores
            df.iloc[i,colindex]=str([df['APLICACAOSVC'][i], df['APLICACAOMNB'][i]])
            continue # passa para o próximo i
        elif df['APLICACAO'][i]==df['APLICACAOSVC'][i]:
            df.iloc[i,colindex]=df['APLICACAOSVC'][i]
            continue # passa para o próximo i
        elif df['APLICACAO'][i]==df['APLICACAOMNB'][i]:
            df.iloc[i,colindex]=df['APLICACAOMNB'][i]
            continue # passa para o próximo i
        elif "," in df['APLICACAO'][i]: # se tiver vírgula, a APLICACAO é uma lista
            aplictemp=df['APLICACAO'][i].replace("[","").replace(", ",";").replace(",",";").replace("'","").replace("]","")
            aplictemp=aplictemp.split(';')
            # se APLICACAOSVC estiver na lista
            if df['APLICACAOSVC'][i] in aplictemp:
                df.iloc[i,colindex]=df['APLICACAOSVC'][i]
                continue # passa para o próximo i
            # se APLICACAOMNB estiver na lista
            elif df['APLICACAOMNB'][i] in aplictemp:
                df.iloc[i,colindex]=df['APLICACAOMNB'][i]
                continue # passa para o próximo i
            else:
                aplictemp.append(df['APLICACAOSVC'][i])
                aplictemp.append(df['APLICACAOMNB'][i])
                df.iloc[i,colindex]=str(aplictemp)
                continue # passa para o próximo i
        else: # caso seja um valor (diferente de ambos os classificadores)
            # APLICACAO!=APLICACAOSVC!=APLICACAOMNB ==> APLICACAOFIM=[APLICACAO,APLICACAOSVC,APLICACAOMNB]
            df.iloc[i,colindex]=str([df['APLICACAO'][i],df['APLICACAOSVC'][i], df['APLICACAOMNB'][i]])
            continue # passa para o próximo i

### Recomparação das classificações

In [39]:
# Quantidade de registros divergentes entre os modelos SVC e Multinomial NB
df[df['APLICACAOSVC']!=df['APLICACAOMNB']].shape

(1112, 31)

In [40]:
# Quantidade de registros divergentes entre a extração e o modelo SVC
df[df['APLICACAOFIM']!=df['APLICACAOSVC']].shape

(649, 31)

In [41]:
# Quantidade de registros divergentes entre a extração e o modelo SVC
df[df['APLICACAOFIM']!=df['APLICACAOMNB']].shape

(620, 31)

In [42]:
# Registros ainda não definidos (lista de opções)
# O filtro definirá True se APLICACAOFIM iniciar com '[' ou False caso contrário.
filtro=[]
for i, aplicacao in enumerate(df['APLICACAOFIM']):
    if aplicacao[0]=='[':
        filtro.append(True)
    else:
        filtro.append(False)

In [43]:
df[filtro].iloc[:,-5:].head()

,Modelo,APLICACAO,APLICACAOSVC,APLICACAOMNB,APLICACAOFIM
11,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER,"['SUNDOWN HUNTER', 'HONDA CT 125 HUNTER']"
16,YAMAHA TENERE XT 250,XXX,YAMAHA XTZ TENERE 250,YAMAHA XT TENERE 660Z 660,"['YAMAHA XTZ TENERE 250', 'YAMAHA XT TENERE 66..."
289,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER,"['SUNDOWN HUNTER', 'HONDA CT 125 HUNTER']"
311,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER,"['SUNDOWN HUNTER', 'HONDA CT 125 HUNTER']"
632,HONDA CB 300 MAX,XXX,SUNDOWN MAX,HONDA CB 300R 300 CB300,"['SUNDOWN MAX', 'HONDA CB 300R 300 CB300']"


In [44]:
df[filtro].shape

(157, 31)

Precisamos agora fazer a observação manual dos registros divergentes para correção.<br>
A seguir, exportaremos o arquivo em excel para fazer a classificação manual em outro notebook.

## Exportando o DataSet para classificação manual

Exportando para um arquivo CSV

In [45]:
df.to_csv(r'./bases/dataframe_modelos_classificado_manual.csv', index = False, header = True)

Exportando para um arquivo de planilha do Excel

In [46]:
df.to_excel(r'./bases/dataframe_modelos_classificado_manual.xlsx', index = False, header = True)